In [179]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.special as special

from sklearn.utils import resample

salaries = pd.read_csv('salaries.csv')
salaries.head()
multiple_testing_pvalues = []

In [180]:
salaries.describe()

,Unnamed: 0,Unnamed: 0.1,Agency,Salary2010,Salary2011,Salary2012,Salary2013
count,52256.000000,52256.000000,52256.000000,5.225600e+04,5.225600e+04,5.225600e+04,5.225600e+04
mean,35504.500593,156430.169205,409.339655,4.938067e+04,5.212639e+04,5.251105e+04,5.030586e+04
std,21453.363830,89673.627737,155.483410,3.391431e+04,3.443960e+04,3.523870e+04,3.672471e+04
min,2.000000,10.000000,11.000000,1.000000e+02,1.000000e+02,1.020000e+02,1.010000e+02
25%,17596.750000,83462.750000,305.000000,3.043250e+04,3.379500e+04,3.392975e+04,3.065450e+04
50%,33986.500000,155251.000000,360.000000,4.632200e+04,4.818700e+04,4.888950e+04,4.709300e+04
75%,52524.250000,225747.500000,490.000000,6.422325e+04,6.604500e+04,6.633725e+04,6.564025e+04
max,79084.000000,339762.000000,696.000000,1.982918e+06,2.529168e+06,2.736431e+06,2.633642e+06


In [181]:
# 1.1

# X = Salary2010 -- random variable
# Null hypothesis #1: mu(X) = mu_1 = 4.91e+04
#  Alt hypothesis #1: mu(x) > mu_1 

mu_1 = 4.91e+04
t_cr = stats.t.ppf(0.95, salaries['Salary2010'].count()-1)
t_obs = (salaries['Salary2010'].mean() - mu_1) / (salaries['Salary2010'].std()
                                                /np.sqrt(salaries['Salary2010'].count()))
#print t_cr
#print t_obs 
pvalue = stats.t.sf(t_obs, salaries['Salary2010'].count())
multiple_testing_pvalues.append(pvalue)

if t_obs > t_cr:
    print "Null hypothesis #1 must be rejected."
else:
    print "Null hypothesis #1 cannot be rejected."

# Null hypothesis #2: mu(X) = mu_2 = 4.95e+04
#  Alt hypothesis #2: mu(x) < mu_2 
mu_2 = 4.95e+04
t_cr = stats.t.ppf(0.05, salaries['Salary2010'].count()-1)
t_obs = (salaries['Salary2010'].mean() - mu_2)/(salaries['Salary2010'].std()
                                                /np.sqrt(salaries['Salary2010'].count()))

#print t_cr
#print t_obs 
pvalue = stats.t.sf(t_obs, salaries['Salary2010'].count())
multiple_testing_pvalues.append(pvalue)

if t_obs < t_cr:
    print "Null hypothesis #2 must be rejected."
else:
    print "Null hypothesis #2 cannot be rejected."

# equivalent library function: 
# stats.ttest_1samp(salaries['Salary2010'], mu_2)

Null hypothesis #1 must be rejected.
Null hypothesis #2 cannot be rejected.


In [182]:
# 1.2 (a) -- parametric test (T-test) -- for 2 INDEPENDENT samples
# If we observe a large p-value, for example larger than 0.05 or 0.1,
# then we cannot reject the null hypothesis of identical average scores.
# Overwise we reject the null hypothesis of equal averages.

# H_0: M1 = M2 -- lets check 2 parts of 'Salary2010'
sz = salaries['Salary2010'].size / 2
stat, pvalue = stats.ttest_ind(salaries['Salary2010'][sz::], salaries['Salary2010'][::sz], equal_var=False)
print pvalue
multiple_testing_pvalues.append(pvalue)
if pvalue > 0.05:
    print "Null hypothesis #3 cannot be rejected."
else:
    print "Null hypothesis #3 must be rejected."
    
# 1.2 (b) -- nonparametric test (Sign test) -- for 2 INDEPENDENT samples
# H_0: median_1 = median_2
stat, pvalue = stats.mannwhitneyu(salaries['Unnamed: 0.1'][sz::], salaries['Unnamed: 0'][::sz])
print pvalue
multiple_testing_pvalues.append(pvalue)
if pvalue > 0.05:
    print "Null hypothesis #4 cannot be rejected."
else:
    print "Null hypothesis #4 must be rejected."

0.487354929065
Null hypothesis #3 cannot be rejected.
0.00715666391959
Null hypothesis #4 must be rejected.


In [183]:
# 1.3 (a) T-test for 2 DEPENDENT samples
samp_1 = salaries['Salary2010']
samp_2 = salaries['Salary2013']
dif_samp = samp_1 - samp_2
mean = dif_samp.mean()
std = dif_samp.std()
n = dif_samp.size
df = n - 1
t_cr = stats.t.ppf(0.95, df)
t_obs = mean / (std * np.sqrt(n))

pvalue = stats.t.sf(t_obs, dif_samp.count())
multiple_testing_pvalues.append(pvalue)

if t_obs < t_cr:
    print "Null hypothesis #5 must be rejected."
else:
    print "Null hypothesis #6 cannot be rejected."

# 1.3 (b) Wilcoxon for 2 DEPENDENT samples
stat, pvalue = stats.wilcoxon(dif_samp)
multiple_testing_pvalues.append(pvalue)
if pvalue > 0.05:
    print "Null hypothesis #6 cannot be rejected."
else:
    print "Null hypothesis #6 must be rejected."

Null hypothesis #5 must be rejected.
Null hypothesis #6 must be rejected.


In [184]:
#2 Power of test
def one_sample_test_power(data, mu_0, rand_sample_size, iters=1000):
    tests=[]
    for j in range(iters):
        rand_samp = data.sample(n=rand_sample_size)
        pvalue = stats.ttest_1samp(rand_samp, mu_0)[1]
        #pvalue = stats.wilcoxon(rand_samp)[1]
        tests.append(pvalue)
        
    tests=np.array(tests)<0.05
    tests = pd.DataFrame(np.mean(tests, axis=0).reshape(-1, 1))
    tests.columns=["Share of H0 rejected:"]
    tests.index=["T-test"]    
    
    return tests

# H_0: mu = mu_0; H_1: mu != mu_0
mu_0 = 4.938e+04
one_sample_test_power(salaries['Salary2010'], 4.938e+04, rand_sample_size=1000)

,Share of H0 rejected:
T-test,0.047


In [185]:
#3 FDR, Bonferroni, Sidak, Bonferroni-Holm
from statsmodels.sandbox.stats.multicomp import multipletests

print '4 hypothesis were rejected.'
multiple_testing_pvalues = np.array(multiple_testing_pvalues)
methods = ['fdr_bh', 'bonferroni', 'sidak', 'holm']

for m in methods:
    is_reject, corrected_pvals, _, _ = multipletests(multiple_testing_pvalues, alpha=0.05, method=m)
    print 'Number of rejections in {}: {}'.format(m, is_reject.sum())

# Results have changed.

4 hypothesis were rejected.
Number of rejections in fdr_bh: 2
Number of rejections in bonferroni: 2
Number of rejections in sidak: 2
Number of rejections in holm: 2


In [191]:
#4 (a) parametric interval for mean
mean_initial = np.mean(salaries['Salary2010'])
errs = []
for i in xrange(1000):
    errs.append(resample(salaries['Salary2010'], replace=True).mean())

print mean_initial
print np.percentile(np.sort(errs), 95)
print np.percentile(np.sort(errs), 5)

49380.6672344
49619.0230825
49141.9102237


In [206]:
#4 (b) nonparametric interval for median
sample = salaries["Unnamed: 0.1"]
median_initial = np.median(sample)
sample_mean = sample.mean()
sample_std  = sample.std()
sample_size = sample.size / 100

errs = []
for i in xrange(1000):
    errs.append(np.median(np.random.normal(sample_mean, sample_std, sample_size)))

print median_initial
print np.percentile(np.sort(errs), 95)
print np.percentile(np.sort(errs), 5)

155251.0
164373.250468
148183.855317
